In [8]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
import skimage
import os
import pickle

nAnchors = 24
batchSize = 16
numEpochs = 3
device = "cuda"

In [9]:
def extract_feats(model, df, fileDir, size=(224,224), outfile=None, nImages=330):
    newDf = df.copy()
    newDf.loc[:,"Features"] = np.zeros((newDf.shape[0], 2208)).tolist()
    newDf.loc[:,"Features"] = newDf.loc[:,"Features"].astype(object)
    
    # define new index for images where first significant digit is
    # sequence number, thus it has to have more overall digits 
    # than max image number. 
    newDf.loc[:,"ImageNum"] = np.zeros(newDf.shape[0])
    indexer = 10**len(str(nImages))
    res = torchvision.transforms.Resize(size)

    for idx, row in df.iterrows():

        name = row["ImageFile"].split(".")[0]

        seq = int(name.split("/")[0].split("q")[-1])
        frame = int(name.split("frame")[-1])

        newDf.loc[idx, "ImageNum"] = indexer*seq + frame

        im = res(torch.transpose(torch.transpose(torch.from_numpy(skimage.io.imread(os.path.join(fileDir,row["ImageFile"]))), 1,2),0,1))

        # extract features and put through GAP layer                                        
        im = im.reshape(1,3,224,224)
        x = model.features(im.float())
        newDf.at[idx,"Features"] = np.squeeze(torch.nn.AvgPool2d(7)(x).detach().numpy())
    
    newDf.sort_values("ImageNum", inplace=True)

    if outfile:
        # use pickle format to make sure values of numpy array
        # are saved with full accuracy (saving as txt only yields
        # around 9 significant digits, pickle saves bit representation)
        newDf.to_pickle(outfile)

    return newDf

def define_anchors(df, nAnchors, outfile=None):
    
    newDf = df.copy()
    newDf.reset_index()
    
    newDf.loc[:,"AnchorDists"] = np.zeros((newDf.shape[0], nAnchors, 2)).tolist()
    newDf.loc[:,"AnchorDists"] = newDf.loc[:,"AnchorDists"].astype(object)

    myAnchors = np.zeros((nAnchors, 2))

    for i, j in enumerate(np.floor(np.linspace(0,df.shape[0], nAnchors, endpoint=False)).astype(int)):
        myAnchors[i,:] = np.array([newDf.loc[j, "X"], newDf.loc[j,"Y"]])
    
    for index, row in newDf.iterrows():
        newDf.at[index, "AnchorDists"] = myAnchors - np.array([newDf.loc[index, "X"], newDf.loc[index,"Y"]])

    if outfile:
        newDf.to_pickle(outfile)

    return newDf, myAnchors

def anchors_for_testSet(df, myAnchors, outfile=None):
    # since anchors are only defined on trainings set
    newDf = df.copy()

    newDf.loc[:,"AnchorDists"] = np.zeros((newDf.shape[0], myAnchors.shape[0], 2)).tolist()
    newDf.loc[:,"AnchorDists"] = newDf.loc[:,"AnchorDists"].astype(object)

    for index, row in newDf.iterrows():
        newDf.at[index, "AnchorDists"] = myAnchors - np.array([newDf.loc[index, "X"], newDf.loc[index,"Y"]])

    if outfile:
        newDf.to_pickle(outfile)

    return newDf

In [3]:
# we use pretrained denseNet
myModel = torchvision.models.densenet161(pretrained=True).float()

In [10]:
trainData = pd.read_csv("data/ShopFacade/dataset_train.txt", delimiter=" ", skiprows=1)
trainData = pd.DataFrame(data={"ImageFile": trainData.loc[:,"ImageFile,"],
                                  "X": trainData.Camera,
                                  "Y": trainData.Position,
                                  "Z": trainData.loc[:,"[X"],
                                  "W": trainData.Y,
                                  "P": trainData.Z,
                                  "Q": trainData.W,
                                  "R": trainData.P})

trainData = extract_feats(myModel, trainData, "data/ShopFacade/", outfile="data/ShopFacade/traindata_with_features.pkl")
trainData, anch = define_anchors(trainData, nAnchors, f"data/ShopFacade/traindata_with_features_and_anchors{nAnchors}.pkl")
np.savetxt(f"data/ShopFacade/anchors{nAnchors}.txt", anch)

In [7]:
testData = pd.read_csv("data/ShopFacade/dataset_test.txt", delimiter=" ", skiprows=1)
testData = pd.DataFrame(data={"ImageFile": testData.loc[:,"ImageFile,"],
                                  "X": testData.Camera,
                                  "Y": testData.Position,
                                  "Z": testData.loc[:,"[X"],
                                  "W": testData.Y,
                                  "P": testData.Z,
                                  "Q": testData.W,
                                  "R": testData.P})

testData = extract_feats(myModel, testData, "data/ShopFacade/", outfile="data/ShopFacade/testdata_with_features.pkl")
testData = anchors_for_testSet(testData, anch, outfile=f"data/ShopFacade/testdata_with_features_and_anchors{nAnchors}.pkl")